# 0 - Information

# 1 - Packages

In [165]:
# Maths packages
import numpy as np

# Dataset packages
import pandas as pd

# Import os packages
from os import listdir
from os.path import isfile, join
import re

# Import progress bar
from tqdm import tqdm

# 2 - Read the Data

In [166]:
def readData(filename):
    """Return the data of the file filename and save them as a numpy array."""
    
    # Resulting dct
    dct = {}
    
    with open(filename) as f:
        
        # Read the first line
        R, C, L, H = f.readline().split()
        
        # Append dct
        dct["R"] = R
        dct["C"] = C
        dct["L"] = L 
        dct["H"] = H
        
        # Resulting dct
        dct["Grid"] = []
        
        # Read the other lines and save them in 
        for line in f:
            dct["Grid"].append(line.strip())
            
    # Convert dct["Data"] as a numpy array
    dct["Grid"] = np.array(dct["Grid"])

    return dct

In [167]:
def readFiles(folder):
    """Read the data in folder and save them in a dict."""
    
    # Resulting dict
    result_dct = {}
    
    # List of files in folder
    files = [f for f in listdir(folder) if isfile(join(folder, f))]
    
    # Loop over all the files in folder
    for file in files:
        
        # Read the data and save them in dct
        file_name = re.sub("\...", "", file)
        result_dct[file_name] = readData(join(folder, file))
    
    return result_dct

In [168]:
# Read the data
data_dct = readFiles("Data")

# Extract one example
test_dct = {"test": data_dct["b"]}

# 3 - Code for the optimisation

In [169]:
def prediction(dataset_dct):
    """Make prediction for the current dataset_dct."""
    
    # Resulting array
    results = []
    
    # Extract R, C, L, H, data
    R = int(dataset_dct["R"])
    C = int(dataset_dct["C"])
    L = int(dataset_dct["L"])
    H = int(dataset_dct["H"])
    grid = dataset_dct["Grid"]
    
    # Make the predictions
    for r in range(R):
        
        # Counters
        beg = 0
        c = 0
        mushroom_count = 0
        tomato_count = 0
        
        # Loop over all column
        while c < C:
            if grid[r][c] == "M":
                mushroom_count += 1
            elif grid[r][c] == "T":
                tomato_count += 1
            
            if c - beg > H:
                if grid[r][beg] == "M":
                    mushroom_count -= 1
                elif grid[r][beg] == "T":
                    tomato_count -= 1
                beg += 1
            
            if (c - beg) < H and (mushroom_count >= L) and (tomato_count >= L):
                results.append(str(r) + " " + str(beg) + " " + str(r) + " " + str(c))
                mushroom_count = 0
                tomato_count = 0
                beg = c + 1
                c = c + 1
            
            # Update of c
            c += 1
                
    return results

In [170]:
def predictionsDct(data_dct):
    """Extract the data of each file in dct and compute the predictions."""
    
    # Resulting dct
    predictions_dct = {}
    
    # Loop over the different datasets
    for key in data_dct.keys():
        
        # Compute the predictions for the current datasets
        predictions_dct[key] = prediction(data_dct[key])
        
    # Return the predictions made
    return predictions_dct

In [171]:
# Compute the predictions
predictions_dct = predictionsDct(data_dct)

# 4 - Save the predictions

In [172]:
def writePredictions(predictions_dct, folder="Results/"):
    """Save the predictions."""
    
    # Loop over all the predictions save in array_dct
    for key in predictions_dct.keys():
        
        # Extract predictions for the given files
        slices = predictions_dct[key]

        # Count the number of slices
        nb_slices = len(slices)
        
        # Writes the result in a txt file
        f = open(folder + key + ".txt", "w")
        
        # Write number of slices
        f.write(str(nb_slices) + "\n")
        
        # Loop over each slices
        for i in slices:
            f.write(i + "\n")
            
        # Closing the file
        f.close()

In [173]:
writePredictions(predictions_dct)